# This Notebook used to scrap tweets and make rule-based sentiment analysis

In [4]:
import tweepy
import json
import numpy as np
import pandas as pd

Insert Twitter API credentials and authenticate. 

In [5]:
ACCESS_TOKEN =    'xxx'
ACCESS_SECRET =   'xxx'
CONSUMER_KEY =    'xxx'
CONSUMER_SECRET = 'xxx'

# auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

"""
REST search API limits 180 Requests/15 mins limit,and per request maximum 100 tweets 
(total limit of 18000 tweets/15mins).
To overvome this limit we will be using Application only Auth instead of the Access Token Auth.
Limits up to 450 request/sec, max 100 tweets/request => rate of 45000 tweets/15min.
"""

auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)


SSLError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /oauth2/token (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

# PART 1: Search REST API

Using the credentials, we can now query using the Twitter's Search Rest API. 
Streaming API could be used for other cases when start monitoring beforehead/inline with event.

Note: With a basic API account, the search results are only from the last 7 days. 
With Premium/Enterprise API - from 2006.

In [3]:
rest_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
#Tweepy will call auto wait (sleep) when reaches rate limit and continue upon expiry of the window

if (not rest_api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [4]:
#make rest-api single query for check (limit 100 twits / query)

#inp = input("Input Company name: ")
inp = 'Deloitte'  #Could be complicated 'Apple OR Google', etc.

raw_results=rest_api.search(q=inp,tweet_mode='extended',count=200)


len(raw_results)  #check number of results

100

To check search results and what usable data a tweet has, we will review the first search result. 

In [5]:
#1. Display _JSON of all received data in first result.
raw_results[0]._json

{'created_at': 'Mon Sep 16 07:45:31 +0000 2019',
 'id': 1173503163644698625,
 'id_str': '1173503163644698625',
 'full_text': 'RT @Lasius_B_Uisso: Deloitte Cyber set out to gain insights into how leaders are embracing the concept of “cyber everywhere;” how they are…',
 'truncated': False,
 'display_text_range': [0, 139],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'Lasius_B_Uisso',
    'name': 'Bullé Uisso',
    'id': 978650672,
    'id_str': '978650672',
    'indices': [3, 18]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 2841824063,
  'id_str': '2841824063',
  'name': 'uwera doreen',
  'screen_name': 'udoreen18',
  'location': 'Rwanda',
  'descr

In [6]:
#2: Use tweepy args to get necessary data
tweet=raw_results[0]
print("@%s - %s - %d followers"%(tweet.user.screen_name, tweet.created_at,tweet.user.followers_count))
print("\nText:%s" % tweet.full_text)
print("Hashtags:%s" % (tweet.entities.get('hashtags')))
print("\nRetweets:%d" % (tweet.retweet_count))
print("Favorites:%d" % (tweet.favorite_count))


@udoreen18 - 2019-09-16 07:45:31 - 643 followers

Text:RT @Lasius_B_Uisso: Deloitte Cyber set out to gain insights into how leaders are embracing the concept of “cyber everywhere;” how they are…
Hashtags:[]

Retweets:1
Favorites:0


### Get amount of data to use it for the purpose of analysis. 
Maintain search context across API limit window, so as to avoid getting duplicates when searching repeatedly over a long period of time we can use:
#1. Using tweepy Cursor (risk of memory leak and crash)
#2. Using manual loop

In [7]:
#1 Using tweepy Cursor
from tqdm import tqdm
results = []
N = 500

#Get the first N items based on the search query    
#For sentiment analysis need to set lang='en', other langs could be always 'neutral'
for tweet in tqdm(tweepy.Cursor(rest_api.search, lang='en', tweet_mode='extended', q=inp).items(N)):  
    results.append(tweet)

# Verify the number of items returned
print(len(results))

500it [00:20, 24.00it/s]


500


In [3]:
# #2 replace tweepy Cursor by manual loop
# api = rest_api
# searchQuery = inp
# maxTweets = 500
# tweetsPerQry = 100  # max API permits



# # If results from a specific ID onwards are reqd, set since_id to that ID.
# # else default to no lower limit, go as far back as API allows
# sinceId = None

# # If results only below a specific ID are, set max_id to that ID.
# # else default to no upper limit, start from the most recent tweet matching the search query.
# max_id = -1

# tweetCount = 0
# tweets = []
# print("Downloading max {0} tweets".format(maxTweets))

# while tweetCount < maxTweets:
#     try:
#         if (max_id <= 0):
#             if (not sinceId):
#                 new_tweets = api.search(q=searchQuery, lang='en', tweet_mode='extended', count=tweetsPerQry)
#             else:
#                 new_tweets = api.search(q=searchQuery, lang='en', tweet_mode='extended', count=tweetsPerQry,
#                                         since_id=sinceId)
#         else:
#             if (not sinceId):
#                 new_tweets = api.search(q=searchQuery, lang='en', tweet_mode='extended', count=tweetsPerQry,
#                                         max_id=str(max_id - 1))
#             else:
#                 new_tweets = api.search(q=searchQuery, lang='en', tweet_mode='extended', count=tweetsPerQry,
#                                         max_id=str(max_id - 1),
#                                         since_id=sinceId)
#         if not new_tweets:
#             print("No more tweets found")
#             break
#         for tweet in new_tweets:
#             tweets.append(tweet)
#         tweetCount += len(new_tweets)
#         print("Downloaded {0} tweets".format(tweetCount))
#         max_id = new_tweets[-1].id
#     except tweepy.TweepError as e:
#         # Just exit if any error
#         print("some error : " + str(e))
#         break
# print ("Downloaded totally {0} tweets".format(tweetCount))


### Convert the data into a Pandas dataframe

In [9]:
def toDataFrame(tweets):
    dataset = pd.DataFrame()
    dataset['tweetID'] = [tweet.id for tweet in tweets]
    dataset['tweetText'] = [tweet.full_text for tweet in tweets]
    dataset['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    dataset['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    dataset['tweetSource'] = [tweet.source for tweet in tweets]
    dataset['tweetCreated'] = [tweet.created_at for tweet in tweets]
    dataset['userID'] = [tweet.user.id for tweet in tweets]
    dataset['user'] = [tweet.user.screen_name for tweet in tweets]
    dataset['userName'] = [tweet.user.name for tweet in tweets]
    dataset['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    dataset['userDesc'] = [tweet.user.description for tweet in tweets]
    dataset['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    dataset['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    dataset['userLocation'] = [tweet.user.location for tweet in tweets]
    dataset['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    dataset['language'] = [tweet.lang for tweet in tweets]
    dataset['hashtags'] = [tweet.entities['hashtags'] for tweet in tweets]
    return dataset



tweets_df = toDataFrame(results)
tweets_df.head(20)

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,user,userName,userCreateDt,userDesc,userFollowerCt,userFriendsCt,userLocation,userTimezone,language,hashtags
0,1173503163644698625,RT @Lasius_B_Uisso: Deloitte Cyber set out to ...,1,0,Twitter Web App,2019-09-16 07:45:31,2841824063,udoreen18,uwera doreen,2014-10-23 10:15:09,"Auditor, Deloitte & Touche. Toastmaster | Rota...",643,699,Rwanda,None,en,[]
1,1173502689537191938,2019 Future of cyber survey https://t.co/71foE...,0,0,Dynamic Signal,2019-09-16 07:43:38,50047200,Karenflade,Karen Falade,2009-06-23 17:53:25,I know the power to change is within me. In pu...,389,276,West Africa,None,en,[]
2,1173501819072348160,.@Oracle and Deloitte help #B2C and #B2B brand...,0,0,Dynamic Signal,2019-09-16 07:40:10,485008283,msavazzi,Massimo Savazzi,2012-02-06 18:44:28,Helping the market to really understand what C...,184,70,Italy - Milan greater area,None,en,"[{'text': 'B2C', 'indices': [27, 31]}, {'text'..."
3,1173501548296454145,"RT @futureof_work: Skills change, but capabili...",1,0,Twitter Web App,2019-09-16 07:39:06,307670634,WISE_Employment,WISE Employment,2011-05-30 02:37:32,Australian not-for-profit employment services ...,1699,1340,Australia,None,en,[]
4,1173501531812843520,Guide to Fiscal Information - Key Economies in...,0,0,Dynamic Signal,2019-09-16 07:39:02,50047200,Karenflade,Karen Falade,2009-06-23 17:53:25,I know the power to change is within me. In pu...,389,276,West Africa,None,en,[]
5,1173501491925200896,RT @FrankumCaroline: ‘It’s not about being vis...,1,0,Twitter Web App,2019-09-16 07:38:52,950653111756050432,INvolvePeople,INvolve,2018-01-09 08:58:58,"Hi, we’re INvolve, the inclusion people.\nWe’r...",1236,334,"London, England",None,en,[]
6,1173501323502915584,@Aquisteph @FrostWalker2017 @PaulNewland12 @no...,0,0,Twitter for Android,2019-09-16 07:38:12,380505915,CLLo_12,originalClo,2011-09-26 19:22:23,,56,136,,None,en,[]
7,1173501273561358337,"Skills change, but capabilities endure Why fos...",1,0,Buffer,2019-09-16 07:38:00,771036263155630080,futureof_work,Future of Work,2016-08-31 17:25:45,Sharing the essential links about #futureofwor...,2622,3232,Global,None,en,"[{'text': 'futureofwork', 'indices': [164, 177..."
8,1173501026663620613,Busting 3 big myths: will robots take our jobs...,0,0,CoSchedule,2019-09-16 07:37:02,1845615044,GoodDeedsBuyers,Good Deeds,2013-09-09 13:09:49,Veronica Morgan is principal of Good Deeds Pro...,413,311,"Balmain, Sydney, Australia",None,en,"[{'text': 'BuyersAgent', 'indices': [188, 200]}]"
9,1173501024029597696,The future of work and importance of human cap...,0,0,Buffer,2019-09-16 07:37:01,733582118186422272,DiManEx_BV,DiManEx,2016-05-20 08:56:21,DiManEx is a digital supply chain service prov...,136,214,"Utrecht, Nederland",None,en,"[{'text': 'changemanagement', 'indices': [101,..."


### Clean up data

In [10]:
#Get hashtags text from dictionary with indeces:
ht=[]
for s in range(len(tweets_df['hashtags'])):
    hasht=[]
    for t in range(len(tweets_df['hashtags'][s])):
        hasht.append(tweets_df['hashtags'][s][t]['text'])
        t=t+1
    ht.append(hasht)
    s=s+1
tweets_df['hashtags'] = ht
tweets_df.head()

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,user,userName,userCreateDt,userDesc,userFollowerCt,userFriendsCt,userLocation,userTimezone,language,hashtags
0,1173503163644698625,RT @Lasius_B_Uisso: Deloitte Cyber set out to ...,1,0,Twitter Web App,2019-09-16 07:45:31,2841824063,udoreen18,uwera doreen,2014-10-23 10:15:09,"Auditor, Deloitte & Touche. Toastmaster | Rota...",643,699,Rwanda,None,en,[]
1,1173502689537191938,2019 Future of cyber survey https://t.co/71foE...,0,0,Dynamic Signal,2019-09-16 07:43:38,50047200,Karenflade,Karen Falade,2009-06-23 17:53:25,I know the power to change is within me. In pu...,389,276,West Africa,None,en,[]
2,1173501819072348160,.@Oracle and Deloitte help #B2C and #B2B brand...,0,0,Dynamic Signal,2019-09-16 07:40:10,485008283,msavazzi,Massimo Savazzi,2012-02-06 18:44:28,Helping the market to really understand what C...,184,70,Italy - Milan greater area,None,en,"[B2C, B2B, CX, customer, OOW19]"
3,1173501548296454145,"RT @futureof_work: Skills change, but capabili...",1,0,Twitter Web App,2019-09-16 07:39:06,307670634,WISE_Employment,WISE Employment,2011-05-30 02:37:32,Australian not-for-profit employment services ...,1699,1340,Australia,None,en,[]
4,1173501531812843520,Guide to Fiscal Information - Key Economies in...,0,0,Dynamic Signal,2019-09-16 07:39:02,50047200,Karenflade,Karen Falade,2009-06-23 17:53:25,I know the power to change is within me. In pu...,389,276,West Africa,None,en,[]


In [ ]:
#To-Do:
#drop names, numbers, urls.


# Part 2: Sentiment Analysis

### Rule-based Sentim Analysis

In [6]:
#1.vader
"""Hutto, C.J. & Gilbert, E.E. (2014). 
VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. 
Eighth International Conference on Weblogs and Social Media (ICWSM-14). 
Ann Arbor, MI, June 2014.
"""
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\oleksii.yeromenko\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
sid = SentimentIntensityAnalyzer()
tweets_df['Sentim_vader'] = tweets_df['tweetText'].apply(sid.polarity_scores)
tweets_df['Sentim_vader_res'] = tweets_df['Sentim_vader'].apply(lambda x: 'positive' if x['compound']>0 else ('negative' if x['compound']<0 else 'neutral'))
tweets_df.head()

NameError: name 'SentimentIntensityAnalyzer' is not defined

In [13]:
#2. Textblob
from textblob import TextBlob
tweets_df['Sentim_blob_polarity'] = tweets_df['tweetText'].apply(lambda x: TextBlob(x).sentiment.polarity)
tweets_df['Sentim_blob_polarity'] = tweets_df['Sentim_blob_polarity'].apply(lambda x: 'positive' if x>0 else ('negative' if x<0 else 'neutral'))
tweets_df['Sentim_blob_subjectivity'] = tweets_df['tweetText'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
tweets_df['Sentim_blob_subjectivity'] = tweets_df['Sentim_blob_subjectivity'].apply(lambda x: 'subjective' if x>0.5 else 'objective')

tweets_df.head(5)

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,user,userName,userCreateDt,...,userFollowerCt,userFriendsCt,userLocation,userTimezone,language,hashtags,Sentim_vader,Sentim_vader_res,Sentim_blob_polarity,Sentim_blob_subjectivity
0,1173503163644698625,RT @Lasius_B_Uisso: Deloitte Cyber set out to ...,1,0,Twitter Web App,2019-09-16 07:45:31,2841824063,udoreen18,uwera doreen,2014-10-23 10:15:09,...,643,699,Rwanda,None,en,[],"{'neg': 0.0, 'neu': 0.861, 'pos': 0.139, 'comp...",positive,neutral,objective
1,1173502689537191938,2019 Future of cyber survey https://t.co/71foE...,0,0,Dynamic Signal,2019-09-16 07:43:38,50047200,Karenflade,Karen Falade,2009-06-23 17:53:25,...,389,276,West Africa,None,en,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,neutral,objective
2,1173501819072348160,.@Oracle and Deloitte help #B2C and #B2B brand...,0,0,Dynamic Signal,2019-09-16 07:40:10,485008283,msavazzi,Massimo Savazzi,2012-02-06 18:44:28,...,184,70,Italy - Milan greater area,None,en,"[B2C, B2B, CX, customer, OOW19]","{'neg': 0.0, 'neu': 0.802, 'pos': 0.198, 'comp...",positive,positive,objective
3,1173501548296454145,"RT @futureof_work: Skills change, but capabili...",1,0,Twitter Web App,2019-09-16 07:39:06,307670634,WISE_Employment,WISE Employment,2011-05-30 02:37:32,...,1699,1340,Australia,None,en,[],"{'neg': 0.0, 'neu': 0.872, 'pos': 0.128, 'comp...",positive,positive,objective
4,1173501531812843520,Guide to Fiscal Information - Key Economies in...,0,0,Dynamic Signal,2019-09-16 07:39:02,50047200,Karenflade,Karen Falade,2009-06-23 17:53:25,...,389,276,West Africa,None,en,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral,neutral,subjective


In [14]:
#Average sentiment on requested name:
print('Vader score for "{}": {}'.format(inp, tweets_df['Sentim_vader_res'].mode()[0]))
print(tweets_df['Sentim_vader_res'].value_counts())
print('---')
print('TextBlob score for "{}": {}'.format(inp, tweets_df['Sentim_blob_polarity'].mode()[0]))
print(tweets_df['Sentim_blob_polarity'].value_counts())

Vader score for "Deloitte": positive
positive    268
neutral     165
negative     67
Name: Sentim_vader_res, dtype: int64
---
TextBlob score for "Deloitte": positive
positive    237
neutral     235
negative     28
Name: Sentim_blob_polarity, dtype: int64


In [26]:
#Most negative post:
tweets_df.loc[tweets_df['Sentim_vader'].apply(lambda x: x['neg']).idxmax()]['tweetText']

'This is very disappointing  https://t.co/MzdqRZRwgq'

In [28]:
#Most positive post:
tweets_df.loc[tweets_df['Sentim_vader'].apply(lambda x: x['pos']).idxmax()]['tweetText']

'This Joe from Deloitte feeling like fine boy#BBNaijaEviction'

#### To-Do Next:


In [ ]:
#Add weight for tweets in result sentiment  - num of followers/retweets, influencers, etc. 

### 2. Automated Sentim Analysis

In [15]:
#To-Do